In [ ]:
import os
cd = os.getcwd()
print(cd)

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import pandas as pd

from xgboost import XGBClassifier as XGBC
xgb_multi = XGBC(learning_rate=0.1, max_depth=14, n_estimators=132,enable_categorical=True)

from sklearn.ensemble import RandomForestClassifier as RFC
rfc_multi = RFC(max_depth=16, n_estimators=220)

from sklearn.ensemble import AdaBoostClassifier
adb_multi = AdaBoostClassifier(learning_rate=1.8, n_estimators=300)

In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/SHP6004/sph6004_assignment1_data.csv')


In [ ]:
df_copy = df.copy()

In [ ]:
cols_select = ['aki','gender', 'admission_age' ,'sbp_min' ,'dbp_min', 'mbp_max', 'resp_rate_max',
 'temperature_min' ,'temperature_max', 'spo2_min', 'ph_max' ,'totalco2_max',
 'calcium_min' ,'bun_min','ptt_max' ,'height', 'weight_admit']

df = df[cols_select]

In [ ]:
print(df.shape)
df.aki.value_counts()

(50920, 17)


0    16860
2    16288
1     9838
3     7934
Name: aki, dtype: int64

In [ ]:
df.dropna(axis=0,inplace=True)
df.shape

(12514, 17)

In [ ]:
df.aki.value_counts()

2    4847
3    2671
1    2590
0    2406
Name: aki, dtype: int64

In [ ]:
df.dtypes

aki                  int64
gender              object
admission_age      float64
sbp_min            float64
dbp_min            float64
mbp_max            float64
resp_rate_max      float64
temperature_min    float64
temperature_max    float64
spo2_min           float64
ph_max             float64
totalco2_max       float64
calcium_min        float64
bun_min            float64
ptt_max            float64
height             float64
weight_admit       float64
dtype: object

In [ ]:
df.gender.loc[df.gender=='M'] = 1
df.gender.loc[df.gender=='F'] = 0

<ipython-input-14-9d829621d483>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.gender.loc[df.gender=='M'] = 1
<ipython-input-14-9d829621d483>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.gender.loc[df.gender=='F'] = 0


In [ ]:
df.gender.value_counts()

1    8089
0    4425
Name: gender, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
cols = ['aki','gender']
df[cols]=df[cols].astype('category')
df.dtypes

X_df = df.iloc[:, 1:]
y_df = df.aki


X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=12)

In [ ]:
y_train.value_counts()

2    3884
3    2135
1    2039
0    1953
Name: aki, dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTENC

# Use SMOTE to resample minority class.
smote_sampler = SMOTENC(random_state=12,sampling_strategy='minority',categorical_features=[0] )
X_train_SMOTE, y_train_SMOTE = smote_sampler.fit_resample(X_train, y_train)

In [ ]:
y_train_SMOTE.value_counts()

0    3884
2    3884
3    2135
1    2039
Name: aki, dtype: int64

In [ ]:
xgb_multi.fit(X_train, y_train)

rfc_multi.fit(X_train, y_train)

adb_multi.fit(X_train, y_train)

AdaBoostClassifier(learning_rate=1.8, n_estimators=300)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

## Gradient Boosting

In [ ]:
y_xgb = xgb_multi.predict(X_test)

print('F1 score on test set: {:.4f}'.format(f1_score(y_test,y_xgb, average='weighted')))
print('accuracy score on test set: {:.4f}'.format(accuracy_score(y_test,y_xgb)))
pd.crosstab(y_test,y_xgb)

F1 score on test set: 0.4336
accuracy score on test set: 0.4499


col_0,0,1,2,3
aki,,,,
0,132,108,192,21
1,104,107,305,35
2,103,102,624,134
3,28,35,210,263


## Random Forest

In [ ]:
y_rfc = rfc_multi.predict(X_test)

print('F1 score on test set: {:.4f}'.format(f1_score(y_test,y_rfc, average='weighted')))
print('accuracy score on test set: {:.4f}'.format(accuracy_score(y_test,y_rfc)))
pd.crosstab(y_test,y_rfc)

F1 score on test set: 0.4190
accuracy score on test set: 0.4714


col_0,0,1,2,3
aki,,,,
0,157,22,252,22
1,101,22,392,36
2,81,17,743,122
3,13,9,256,258


## AdaBoosting

In [ ]:
y_adb = adb_multi.predict(X_test)

print('F1 score on test set: {:.4f}'.format(f1_score(y_test,y_adb, average='weighted')))
print('accuracy score on test set: {:.4f}'.format(accuracy_score(y_test,y_adb)))
pd.crosstab(y_test,y_adb)

F1 score on test set: 0.4223
accuracy score on test set: 0.4507


col_0,0,1,2,3
aki,,,,
0,178,51,192,32
1,125,56,327,43
2,124,62,639,138
3,40,24,217,255


## Ensemble

In [ ]:
pred_df = pd.DataFrame({'XGB': y_xgb, 'RF': y_rfc, 'AB': y_adb})
pred_df

,XGB,RF,AB
0,1,2,0
1,1,2,0
2,1,2,2
3,0,0,0
4,2,2,2
...,...,...,...
2498,2,2,2
2499,0,0,0
2500,2,2,2
2501,2,2,2


In [ ]:
import random

def get_random_mode(row):
  m = row.mode()
  if (len(m)>1):
    m = random.choice(m)
  return m.squeeze()

In [ ]:
y_pred_en = pred_df.apply(get_random_mode, axis=1)
print('F1 score on test set: {:.4f}'.format(f1_score(y_test,y_pred_en,average='weighted')))
print('accuracy score on test set: {:.4f}'.format(accuracy_score(y_test,y_pred_en)))
pd.crosstab(y_test,y_pred_en)

F1 score on test set: 0.4241
accuracy score on test set: 0.4658


col_0,0,1,2,3
aki,,,,
0,1,0,15,3
1,1,0,17,2
2,6,4,32,8
3,6,1,15,4
